In [5]:
import pandas as pd
import numpy as np
import pandas as pd
import pylab as pl
from sklearn.neighbors import KNeighborsClassifier


In [6]:
def dfplayerYear(year):
    dfplayerYr=pd.read_csv('../Data/draft_year_player/dfplayer'+str(year)+'.csv')
    namePlayer=[]
    
    for elem in dfplayerYr['url']:
        namePlayer.append(elem[11:-5])

    dfplayerYr['name']=namePlayer

    del dfplayerYr['Unnamed: 0']
    del dfplayerYr['url']
    del dfplayerYr['draft']
    del dfplayerYr['shot']
    del dfplayerYr['pos']
    
    return dfplayerYr

In [7]:
fundam = []
for yr in range(1996,2016):
    fundam.append(dfplayerYear(yr))

In [8]:
df_all_fundam = pd.concat(fundam)
df_all_fundam=df_all_fundam.reset_index(drop=True)

df_all_fundam = df_all_fundam[df_all_fundam.height!= 'ml>']
df_all_fundam=df_all_fundam.reset_index(drop=True)
weight_all=df_all_fundam['weight']
height_all=df_all_fundam['height']

df_all_fundam[['weight','height']]=df_all_fundam[['weight','height']].astype(float)

### Height Weight Data

In [12]:
def dfplayerAge(age):
    df_age_yr=pd.read_csv('../Data/AgeData/age_data/df_age_'+str(age)+'.csv',index_col=0)
    return df_age_yr

In [13]:
df_age_frames=[]
for age in range(19,40):
    df_age_frames.append(dfplayerAge(age))

df_ages = pd.concat(df_age_frames)
df_ages=df_ages.reset_index(drop=True)
del df_ages['age_y']


In [14]:
#calculate goals all 
df_ages['goals_all']=df_ages['goals_ev']+df_ages['goals_pp']+df_ages['goals_sh']

#sort by name and age
df_ages_sorted=df_ages.sort(['name', 'age_x'], ascending=[0, 1])

/Users/takehiromatsuzawa/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [15]:
# reset index
df_ages_sorted=df_ages_sorted.reset_index(drop=True)

df_ages_sorted['goals_all']=df_ages_sorted['goals_all']/df_ages_sorted['games_played']
goals_all_data=list(df_ages_sorted['goals_all'])

# you want to predict the next year data
goals_all_data.pop(0)
goals_all_data.append(0)

df_ages_sorted['goals_all']=goals_all_data
df_ages=df_ages_sorted.groupby("name").apply(lambda x: x.iloc[:-1]).reset_index(drop=True)



### Combined Data (All Data)

In [16]:
df_all = pd.merge(df_ages, df_all_fundam, on='name')
df_all=df_all.drop_duplicates()
df_all['assists_all']=df_all['assists_ev']+df_all['assists_pp']+df_all['assists_sh']

df_all['zs_offense_pct']=df_all['zs_offense_pct']/100.0
df_all['shot_pct']=df_all['shot_pct']/100.0

### convert to per game data

In [17]:
per_games=['goals_ev','goals_pp','goals_sh','assists_all','assists_ev','assists_pp','assists_sh','takeaways',
          'hits','giveaways','blocks','shots_attempted']

for per_game in per_games:
    df_all[per_game]=df_all[per_game]/df_all['games_played']

### Normalize Data

In [18]:
df_all_nn=df_all[['goals_all','goals_ev','goals_pp','goals_sh','assists_ev','assists_pp','assists_sh','shots_attempted','age_x','blocks',
                 'giveaways','hits','shot_pct','shots_attempted','takeaways','zs_offense_pct','height','weight']]

In [19]:
cols_to_norm = ['goals_ev','goals_pp','goals_sh','assists_ev','assists_pp','assists_sh','shots_attempted','age_x','blocks','giveaways',
               'hits','shot_pct','shots_attempted','takeaways','zs_offense_pct','height','weight']
df_all_nn=df_all_nn[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.max() - x.min()))

In [20]:
df_all_nn.head()

,goals_ev,goals_pp,goals_sh,assists_ev,assists_pp,assists_sh,shots_attempted,shots_attempted,age_x,blocks,giveaways,hits,shot_pct,shots_attempted,shots_attempted,takeaways,zs_offense_pct,height,weight
0,-0.108210,-0.098756,-0.008203,-0.177544,-0.124635,-0.020940,0.091770,0.091770,-0.300227,-0.204734,-0.136415,0.172519,-0.080687,0.091770,0.091770,0.078597,0.092615,0.046712,0.122539
1,-0.108210,-0.098756,-0.008203,-0.177544,-0.124635,-0.020940,0.045937,0.045937,-0.247595,-0.204734,0.030251,0.029662,-0.080687,0.045937,0.045937,-0.214372,0.159615,0.046712,0.122539
2,-0.048210,-0.098756,-0.008203,-0.117544,-0.124635,-0.020940,-0.016395,-0.016395,-0.194964,-0.093073,-0.089749,0.249662,-0.042687,-0.016395,-0.016395,-0.132341,0.155615,0.046712,0.122539
3,-0.013616,-0.098756,-0.008203,-0.028895,-0.124635,0.046627,0.005060,0.005060,-0.142332,-0.057614,-0.046325,0.178311,-0.026687,0.005060,0.005060,-0.079765,0.080615,0.046712,0.122539
4,-0.009445,-0.098756,-0.008203,-0.004704,-0.124635,-0.020940,-0.053083,-0.053083,-0.089700,-0.059989,-0.078802,0.076399,-0.014687,-0.053083,-0.053083,-0.098631,0.085615,0.046712,0.122539


In [21]:
df_all_nn['goals_all']=df_all['goals_all']

In [22]:
df_all_nn['age_x'] = df_all_nn['age_x'].apply(lambda x: abs(x))

In [23]:
df_all_nn=df_all_nn.reset_index(drop=True)

In [24]:
df_all_nn.head()

,goals_ev,goals_pp,goals_sh,assists_ev,assists_pp,assists_sh,shots_attempted,shots_attempted,age_x,blocks,giveaways,hits,shot_pct,shots_attempted,shots_attempted,takeaways,zs_offense_pct,height,weight,goals_all
0,-0.108210,-0.098756,-0.008203,-0.177544,-0.124635,-0.020940,0.091770,0.091770,0.300227,-0.204734,-0.136415,0.172519,-0.080687,0.091770,0.091770,0.078597,0.092615,0.046712,0.122539,0.000000
1,-0.108210,-0.098756,-0.008203,-0.177544,-0.124635,-0.020940,0.045937,0.045937,0.247595,-0.204734,0.030251,0.029662,-0.080687,0.045937,0.045937,-0.214372,0.159615,0.046712,0.122539,0.060000
2,-0.048210,-0.098756,-0.008203,-0.117544,-0.124635,-0.020940,-0.016395,-0.016395,0.194964,-0.093073,-0.089749,0.249662,-0.042687,-0.016395,-0.016395,-0.132341,0.155615,0.046712,0.122539,0.094595
3,-0.013616,-0.098756,-0.008203,-0.028895,-0.124635,0.046627,0.005060,0.005060,0.142332,-0.057614,-0.046325,0.178311,-0.026687,0.005060,0.005060,-0.079765,0.080615,0.046712,0.122539,0.098765
4,-0.009445,-0.098756,-0.008203,-0.004704,-0.124635,-0.020940,-0.053083,-0.053083,0.089700,-0.059989,-0.078802,0.076399,-0.014687,-0.053083,-0.053083,-0.098631,0.085615,0.046712,0.122539,0.208333


In [25]:
def knn_method(K,repeat,features):
    results_all=[]
    for k in range(1,K,1):
        results=[]
        for n in range(1, repeat, 1):

            test_idx = np.random.uniform(0, 1, len(df_all_nn)) <= 0.8
            train = df_all_nn[test_idx==True]
            test = df_all_nn[test_idx==False]

            clf = KNeighborsClassifier(n_neighbors=k)
            clf.fit(train[features], np.asarray(train['goals_all'], dtype="|S6"))
            preds = clf.predict(test[features])
            test_goals_all=list(test['goals_all'])
            distance=0
            for i in range(len(test_goals_all)):
                #error.append(test_goals_all[i]-float(preds[i]))
                distance += pow((test_goals_all[i]-float(preds[i])), 2)
            accuracy = distance/ float(len(test))
            results.append(accuracy)

        results_all.append([k, sum(results)/len(results)])
        print "Neighbors: %d, Accuracy: %3f" % (k, sum(results)/len(results))
    
    return results_all

### All variables

In [26]:
features = ['goals_ev','goals_pp','goals_sh','assists_ev','assists_pp','assists_sh','shots_attempted','age_x','blocks','giveaways',
               'hits','shot_pct','shots_attempted','takeaways','zs_offense_pct','height','weight']
results_knn=knn_method(10,10,features)

Neighbors: 1, Accuracy: 0.012247
Neighbors: 2, Accuracy: 0.010495
Neighbors: 3, Accuracy: 0.011565
Neighbors: 4, Accuracy: 0.012748
Neighbors: 5, Accuracy: 0.014026
Neighbors: 6, Accuracy: 0.014860
Neighbors: 7, Accuracy: 0.015902
Neighbors: 8, Accuracy: 0.015637
Neighbors: 9, Accuracy: 0.016504


In [27]:
KNN_Error=pd.DataFrame(results_knn,columns=['Neighbors','Error'])

In [28]:
KNN_Error

,Neighbors,Error
0,1,0.012247
1,2,0.010495
2,3,0.011565
3,4,0.012748
4,5,0.014026
5,6,0.014860
6,7,0.015902
7,8,0.015637
8,9,0.016504


In [29]:
pl.plot(KNN_Error.Neighbors, KNN_Error.Error)
pl.title("Error with Number of Neighbors (KNN)")
pl.xlabel('Neighbors')
pl.ylabel('Error')


### Offensive Variables

In [30]:
features = ['goals_ev','goals_pp','goals_sh','assists_ev','assists_pp','assists_sh','shots_attempted','age_x','zs_offense_pct','takeaways','height','weight']
results_knn=knn_method(10,10,features)

Neighbors: 1, Accuracy: 0.013007
Neighbors: 2, Accuracy: 0.011528
Neighbors: 3, Accuracy: 0.012968
Neighbors: 4, Accuracy: 0.014506
Neighbors: 5, Accuracy: 0.015712
Neighbors: 6, Accuracy: 0.016372
Neighbors: 7, Accuracy: 0.016780
Neighbors: 8, Accuracy: 0.016986
Neighbors: 9, Accuracy: 0.018242


In [31]:
KNN_Error=pd.DataFrame(results_knn,columns=['Neighbors','Error'])

In [32]:
KNN_Error

,Neighbors,Error
0,1,0.013007
1,2,0.011528
2,3,0.012968
3,4,0.014506
4,5,0.015712
5,6,0.016372
6,7,0.016780
7,8,0.016986
8,9,0.018242


In [33]:
pl.plot(KNN_Error.Neighbors, KNN_Error.Error)
pl.title("Error with Number of Neighbors (KNN)")
pl.xlabel('Neighbors')
pl.ylabel('Error')
